# Analyze subway turnstile data for the last 6 weeks

In [1]:
# standard initialization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime, timedelta
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import geocoder
import geoplot
import itertools

In [2]:
# set constants
main_path = '/Users/rita/Dropbox/InsightDataScience/Project'
api_key = open(main_path + '/config.py', 'r')
api_key = api_key.read().replace('api_key = ', '').replace('“','').replace('”','')

In [3]:
# Load datasets and append

def load_and_append_txts(dates):
    path = '/Users/rita/Dropbox/InsightDataScience/Project/Data/Turnstile'
    df = pd.DataFrame()
    for date in dates:
        fn = path + '/turnstile_' + str(date) + '.txt'
        df_new = pd.read_csv(fn)
        df = df.append(df_new)
    # preliminary cleaning
    df = df.rename(columns={'ENTRIES' : 'ENTRIES_CUM',
                            'EXITS                                                               ' 
                            : 'EXITS_CUM'})
    return df

df_1 = load_and_append_txts([190202, 190126, 190119, 190112, 190105, 181229])
df_1.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES_CUM,EXITS_CUM
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2019,03:00:00,REGULAR,6922652,2347673
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2019,07:00:00,REGULAR,6922669,2347688
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2019,11:00:00,REGULAR,6922747,2347773
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2019,15:00:00,RECOVR AUD,6922932,2347849
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2019,19:00:00,REGULAR,6923237,2347911


In [4]:
# Filter to outer borough stations
nyc_subway_map = pd.read_csv(main_path + '/Data/Turnstile/Stations.csv')
nyc_borough_map = gpd.read_file(main_path + '/Data/Locations/Boroughs/geo_export_042a7af9-2558-430b-b512-5799557e5b47.shp')
nyc_subway_trnstl_template =  pd.read_csv(main_path + '/Data/Turnstile/template.txt').loc[:,['STATION', 
                                            'LINENAME']].drop_duplicates().reset_index()

def strip_station_names(s):
    s = s.lower()
    s = s.replace(' ', '')  
    return s
    
nyc_subway_trnstl_template['StationStrip'] = nyc_subway_trnstl_template.STATION.map(strip_station_names)

nyc_subway_map['StationStrip'] = nyc_subway_map['Stop Name'].map(strip_station_names)
nyc_subway_map['LINENAME'] = nyc_subway_map['Daytime Routes'].map(lambda s: s.replace(' ', ''))

# now match stations to get best estimate of lat/long
def match_station_names(s):
    station_name_list = nyc_subway_map.STATION_LINE
    new_name = process.extractOne(s, station_name_list) # scorer=fuzz.partial_token_set_ratio you could change
    return new_name[0]

nyc_subway_trnstl_template['STATION_LINE'] = nyc_subway_trnstl_template.StationStrip + \
                                                nyc_subway_trnstl_template.LINENAME
nyc_subway_map['STATION_LINE'] = nyc_subway_map.StationStrip + nyc_subway_map.LINENAME

nyc_subway_trnstl_template['StationNameMatch'] = nyc_subway_trnstl_template.STATION_LINE.map(match_station_names) # not sure the best way to do this

# get lat/long for each turnstile data subway station.
nyc_subway_trnstl_template = pd.merge(nyc_subway_trnstl_template, nyc_subway_map[['Borough', 'GTFS Latitude', 
                                        'GTFS Longitude', 'STATION_LINE']], 
                                        left_on='StationNameMatch', 
                                        right_on='STATION_LINE').drop(['index', 'STATION_LINE_y', 
                                        'StationStrip'], axis=1).rename(columns={'STATION_LINE_x':
                                        'STATION_LINE_A', 'StationNameMatch': 'STATION_LINE_B', 
                                        'STATION': 'STATION_NAME'})
# Save Matchup to visually inspect
nyc_subway_trnstl_template.to_csv(main_path + '/Results/subway_name_match_check.csv')

# Perform Matchup corrections
matchups_to_correct = pd.read_csv(main_path + '/Results/Matchup_Corrected.csv')
for i in range(0, matchups_to_correct.shape[0]):
    matched_val = nyc_subway_trnstl_template.STATION_LINE_A == matchups_to_correct.loc[i, 'STATION_LINE_A']
    nyc_subway_trnstl_template.loc[matched_val, 'STATION_LINE_B'] = matchups_to_correct.loc[i, 'STATION_LINE_B']
    nyc_subway_trnstl_template.loc[matched_val, 'Borough'] = matchups_to_correct.loc[i, 'Borough']
    nyc_subway_trnstl_template.loc[matched_val, 'GTFS Latitude'] = np.float(matchups_to_correct.loc[i, 'GTFS Latitude'])
    nyc_subway_trnstl_template.loc[matched_val, 'GTFS Longitude'] = np.float(matchups_to_correct.loc[i, 'GTFS Longitude'])


In [5]:
# restrict to outer boroughs
nyc_subway_trnstl_template = nyc_subway_trnstl_template[nyc_subway_trnstl_template.Borough.isin(['Bx', 'Bk',
                                                        'Q', 'SI'])].reset_index(drop=True)
nyc_subway_map = nyc_subway_map[nyc_subway_map.Borough.isin(['Bx', 'Bk', 'Q', 
                                            'SI'])].reset_index(drop=True)

# changes station name to account for stations that are named the same but
# have different lines
nyc_subway_trnstl_template['STATION'] = nyc_subway_trnstl_template['STATION_NAME'] + ' (' \
                                            + nyc_subway_trnstl_template['LINENAME'] + ')'
nyc_subway_trnstl_template = nyc_subway_trnstl_template.drop_duplicates()

df_1 = df_1.rename(columns={'STATION': 'STATION_NAME'})
df_1['STATION'] = df_1['STATION_NAME'] + ' (' + df_1['LINENAME'] + ')'

nyc_subway_map = nyc_subway_map.drop(['Station ID', 'Complex ID', 
                                      'GTFS Stop ID'], axis=1).drop_duplicates(subset=['STATION_LINE'], 
                                      keep=False)

In [6]:
# Select Stations of Interest (remove manhattan)
df_1 = df_1[df_1['STATION'].isin(nyc_subway_trnstl_template.STATION)]

# Filter to STD Times & convert to usable format
std_times = [str(t).zfill(2) + ':00:00' for t in (range(0, 24, 1))] # zfill zero-pads
df_1 = df_1[df_1['TIME'].isin(std_times)]
df_1['TIME'] = df_1['TIME'].map(lambda x: 
                                  int(x.replace(':00', '')))

In [7]:
# check timing counts
df_1.groupby('TIME').count().head()

,C/A,UNIT,SCP,STATION_NAME,LINENAME,DIVISION,DATE,DESC,ENTRIES_CUM,EXITS_CUM,STATION
TIME,,,,,,,,,,,
0,40021,40021,40021,40021,40021,40021,40021,40021,40021,40021,40021
1,294,294,294,294,294,294,294,294,294,294,294
3,55051,55051,55051,55051,55051,55051,55051,55051,55051,55051,55051
4,40014,40014,40014,40014,40014,40014,40014,40014,40014,40014,40014
5,287,287,287,287,287,287,287,287,287,287,287


In [8]:
# Unroll cumulative entries and exits

# Very few entries indicates a station closing
# Negatives represent counter resets
# There are a few unrealistically large values
# REL: Rather than hard code as > 1500, remove outliers
# relative to the station
# Replace with NaN
# REL: Reconsider this approach

def outlier_removal(df, column_name, group_by_column):
    df = df.set_value((df[column_name] <= 0) | (df[column_name] >= 2000), column_name, np.nan)
    quartiles = df.groupby(group_by_column).quantile([0.25, 0.5, 0.75]).reset_index()
    all_metrics = pd.DataFrame({group_by_column: quartiles[group_by_column].unique()})
    all_metrics['MEAN'] = quartiles[quartiles.level_1 == 0.5].reset_index()[column_name]
    all_metrics['SIG'] = 0.74 * (quartiles[quartiles.level_1 == 0.75].reset_index()[column_name] -
                                quartiles[quartiles.level_1 == 0.25].reset_index()[column_name] )
    all_metrics['MIN']= all_metrics['MEAN'] - 5 * all_metrics['SIG']
    all_metrics['MAX']= all_metrics['MEAN'] + 5 * all_metrics['SIG']
    df = pd.merge(df, all_metrics, on=group_by_column, how='inner')
    df.loc[(df[column_name] < df['MIN']) | (df[column_name] > df['MAX']), column_name] = np.nan # sigma clipping
    return df.drop(['MIN', 'MAX', 'MEAN', 'SIG'], axis = 1)

# sort first
df_1 = df_1.sort_values(by=['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'TIME'])

# get non-cummulative
df_1['ENTRIES'] = df_1.groupby(['C/A', 'UNIT', 'SCP', 'STATION'])['ENTRIES_CUM'].diff()
df_1['EXITS'] = df_1.groupby(['C/A', 'UNIT', 'SCP', 'STATION'])['EXITS_CUM'].diff()

# remove outliers
df_1 = outlier_removal(df_1, 'ENTRIES', 'STATION')
df_1 = outlier_removal(df_1, 'EXITS', 'STATION')

/Users/rita/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':


In [9]:
# Slice time correct

temp = pd.DataFrame({'placeholder': [-3, -2, -1, 0]}) # setup for making missing time points
df_1['TIME_INT'] = df_1.groupby(['C/A', 'UNIT', 'SCP', 'STATION'])['TIME'].diff()
df_1 = (df_1.assign(key=1).merge(temp.assign(key=1), on="key").drop("key", axis=1))

# first measure the next day appears negative
df_1.loc[df_1['TIME_INT'] == -20, 'TIME_INT'] = 4 
df_1.loc[df_1['TIME_INT'] == -21, 'TIME_INT'] = 3 
df_1.loc[df_1['TIME_INT'] == -22, 'TIME_INT'] = 2 
df_1.loc[df_1['TIME_INT'] == -23, 'TIME_INT'] = 1 

# drop intervals that are too long
df_1.loc[(df_1['TIME_INT'] < 0), 'TIME_INT'] = np.nan 
df_1.loc[(df_1['TIME_INT'] > 4), 'TIME_INT'] = np.nan

# get values for each hour
df_1['ENTRIES_ST'] = df_1['ENTRIES'] / df_1['TIME_INT'] 
df_1['EXITS_ST'] = df_1['EXITS'] / df_1['TIME_INT'] 

# add them in
df_1['TIME_ST'] = df_1['TIME'] + df_1['placeholder']

# remove duplicate columns
to_remove = ((df_1['placeholder'] == -3) & (df_1['TIME_INT'] == 3)) | \
            ((df_1['placeholder'] <= -2) & (df_1['TIME_INT'] == 2)) | \
            ((df_1['placeholder'] <= -1) & (df_1['TIME_INT'] == 1))  
df_1 = df_1[~to_remove]

# change first entry of the day to last entry of previous day
to_change = df_1['TIME_ST'] <= 0
df_1.loc[to_change, 'DATE'] = df_1.loc[to_change, 'DATE'].map(lambda x: datetime.strftime(datetime.strptime(x, 
                                                                 '%m/%d/%Y') -  timedelta(days=1),'%m/%d/%Y'))
df_1.loc[df_1['TIME_ST'] == 0, 'TIME_ST'] = 24
df_1.loc[df_1['TIME_ST'] == -1, 'TIME_ST'] = 23
df_1.loc[df_1['TIME_ST'] == -2, 'TIME_ST'] = 22
df_1.loc[df_1['TIME_ST'] == -3, 'TIME_ST'] = 21
df_1 = df_1.sort_values(by=['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'TIME_ST'])

#.strftime('%m/%d/%Y') to reformat datetime
df_1['WK_DAY'] = df_1['DATE'].map(lambda x: 
                                  datetime.weekday(datetime.strptime(x, '%m/%d/%Y')))

df_1['WK_DAY_STR'] = df_1['DATE'].map(lambda x: 
                                   datetime.strptime(x, '%m/%d/%Y').strftime("%A"))

# Add an entry/exit count column 
df_1['ENTRIES_CNT'] = df_1['ENTRIES'] / df_1['ENTRIES']
df_1['EXITS_CNT'] = df_1['EXITS'] / df_1['EXITS']

df_1 = df_1.drop(['DESC', 'ENTRIES_CUM', 'EXITS_CUM', 'TIME_INT', 'TIME', 'ENTRIES',
                'EXITS', 'placeholder'], axis=1).rename(columns={'ENTRIES_ST': 'ENTRIES',
                'EXITS_ST': 'EXITS', 'TIME_ST': 'TIME'})

In [10]:
# Group by weekday
df_1_by_wkday = df_1.groupby(['STATION', 'WK_DAY', 'WK_DAY_STR', 'TIME']).agg({'ENTRIES':'mean', 
                             'EXITS':'mean', 'ENTRIES_CNT':'sum', 'EXITS_CNT':'sum'}).reset_index()

# group wkdays by type
df_1.loc[df_1['WK_DAY'] < 4, 'WK_DAY_TYPE'] = 1
df_1.loc[df_1['WK_DAY'] == 4, 'WK_DAY_TYPE'] = 2
df_1.loc[df_1['WK_DAY'] == 5, 'WK_DAY_TYPE'] = 3
df_1.loc[df_1['WK_DAY'] == 6, 'WK_DAY_TYPE'] = 4
df_1_by_wkday_type = df_1.groupby(['STATION', 'WK_DAY_TYPE', 'TIME']).agg({'ENTRIES':'mean', 
                                'EXITS':'mean', 'ENTRIES_CNT':'sum', 'EXITS_CNT':'sum'}).reset_index()

# re-collapse so data doesn't look funny
def agg_time_slots(df):
    df.loc[df['TIME'] <= 4, 'TIME_AGG'] = 1
    df.loc[(df['TIME'] > 4) & (df['TIME'] <= 8), 'TIME_AGG'] = 2
    df.loc[(df['TIME'] > 8) & (df['TIME'] <= 12), 'TIME_AGG'] = 3
    df.loc[(df['TIME'] > 12) & (df['TIME'] <= 16), 'TIME_AGG'] = 4
    df.loc[(df['TIME'] > 16) & (df['TIME'] <= 20), 'TIME_AGG'] = 5
    df.loc[(df['TIME'] > 20) & (df['TIME'] <= 24), 'TIME_AGG'] = 6
    return df

df_1_by_wkday = agg_time_slots(df_1_by_wkday)
df_1_by_wkday = df_1_by_wkday.groupby(['STATION', 'WK_DAY', 'WK_DAY_STR', 'TIME_AGG']).sum().loc[:,['ENTRIES',
                    'EXITS']].sort_values(by=['STATION',
                    'WK_DAY']).reset_index().rename(columns={'TIME_AGG':'TIME'})

df_1_by_wkday_type = agg_time_slots(df_1_by_wkday_type)
df_1_by_wkday_type = df_1_by_wkday_type.groupby(['STATION', 'WK_DAY_TYPE', 'TIME_AGG']).sum().loc[:,['ENTRIES',
                    'EXITS']].sort_values(by=['STATION',
                    'WK_DAY_TYPE']).reset_index().rename(columns={'TIME_AGG':'TIME'})

In [11]:
# remove stations with NaNs
to_keep = df_1_by_wkday_type.groupby(['STATION']).count().reset_index().loc[:,['STATION', 'ENTRIES']]
to_keep = to_keep.rename(columns={'ENTRIES': 'COUNT'})
df_1_by_wkday_type = df_1_by_wkday_type.merge(to_keep, on='STATION')
df_1_by_wkday_type = df_1_by_wkday_type[df_1_by_wkday_type['COUNT'] == 24]

to_keep = df_1_by_wkday.groupby(['STATION']).count().reset_index().loc[:,['STATION', 'ENTRIES']]
to_keep = to_keep.rename(columns={'ENTRIES': 'COUNT'})
df_1_by_wkday = df_1_by_wkday.merge(to_keep, on='STATION')
df_1_by_wkday = df_1_by_wkday[df_1_by_wkday['COUNT'] == 42]

In [12]:
# Standardize data
def standardize_vals(df):
    mean_df = df.groupby(['STATION']).mean().rename(columns={"ENTRIES": "ENTRIES_MEAN", 
                                                                    "EXITS": "EXITS_MEAN"}).reset_index().loc[:,
                                                                    ['STATION', 'ENTRIES_MEAN', 'EXITS_MEAN']]
    sdev_df = df.groupby(['STATION']).std().rename(columns={"ENTRIES": "ENTRIES_SDEV", 
                                                                    "EXITS": "EXITS_SDEV"}).reset_index().loc[:,
                                                                    ['STATION', 'ENTRIES_SDEV', 'EXITS_SDEV']]
    metrics_df = pd.concat([mean_df, sdev_df], axis=1)
    metrics_df = metrics_df.loc[:,~metrics_df.columns.duplicated()]
    df = pd.merge(df, metrics_df, on='STATION', how='inner')
    df['ENTRIES'] = (df['ENTRIES'] - df['ENTRIES_MEAN']) / df['ENTRIES_SDEV']
    df['EXITS'] = (df['EXITS'] - df['EXITS_MEAN']) / df['EXITS_SDEV']
    df = df.drop(['ENTRIES_MEAN', 'ENTRIES_SDEV', 'EXITS_MEAN', 'EXITS_SDEV'], axis=1)
    return df

df_1_by_wkday = standardize_vals(df_1_by_wkday)
df_1_by_wkday_type = standardize_vals(df_1_by_wkday_type)

In [13]:
# get rid of any stations with nulls
df_1_by_wkday['NA_COUNT'] = 0
df_1_by_wkday.loc[df_1_by_wkday.isna()['ENTRIES'], 'NA_COUNT'] += 1
df_1_by_wkday.loc[df_1_by_wkday.isna()['ENTRIES'], 'NA_COUNT'] += 1
to_drop = df_1_by_wkday[df_1_by_wkday.NA_COUNT > 0].STATION.unique()
df_1_by_wkday = df_1_by_wkday[~df_1_by_wkday.STATION.isin(to_drop)]

df_1_by_wkday_type['NA_COUNT'] = 0
df_1_by_wkday_type.loc[df_1_by_wkday_type.isna()['ENTRIES'], 'NA_COUNT'] += 1
df_1_by_wkday_type.loc[df_1_by_wkday_type.isna()['ENTRIES'], 'NA_COUNT'] += 1
to_drop = df_1_by_wkday_type[df_1_by_wkday_type.NA_COUNT > 0].STATION.unique()
df_1_by_wkday_type = df_1_by_wkday_type[~df_1_by_wkday_type.STATION.isin(to_drop)]

In [14]:
# get neighborhood info

# get map of Brooklyn neighborhoods
nyc_nbhd_map = gpd.read_file(main_path + '/Data/Locations/ZillowNeighborhoods-NY/ZillowNeighborhoods-NY.shp')
nyc_subway_trnstl_template['geometry'] = nyc_subway_trnstl_template.apply(lambda row: 
                                                  Point(row['GTFS Longitude'], row['GTFS Latitude']), axis=1)
nyc_subway_trnstl_template = gpd.GeoDataFrame(nyc_subway_trnstl_template, geometry='geometry', crs=nyc_nbhd_map.crs)

nyc_subway_trnstl_template = gpd.tools.sjoin(nyc_subway_trnstl_template, nyc_nbhd_map, how='left').set_index('STATION')
nyc_subway_trnstl_template = nyc_subway_trnstl_template[nyc_subway_trnstl_template.County != 'New York']

In [17]:
def plot_data_by_station_normed(df, station, nbhd_df, folder_name):              
    station_df = df.loc[station][['ENTRIES', 'EXITS']]
    
    nbhd = nbhd_df.loc[station]['Name']
    plot_name = station + ': ' + nbhd
    
    font = {'family' : 'normal',
            'weight' : 'medium',
            'size'   : 16}
    matplotlib.rc('font', **font)
      
    s = pd.Series(np.arange(1,40))
    station_df = station_df.reset_index(drop=True)
    
    fig, ax = plt.subplots(1, figsize=(12,5))
    matplotlib.pyplot.title(plot_name, fontsize=20, fontweight='heavy')

    station_df.plot(ax=ax, marker='o')

    ax.vlines(0, -2.5, 2.5)
    ax.vlines(6, -2.5, 2.5)
    ax.vlines(12, -2.5, 2.5)
    ax.vlines(18, -2.5, 2.5)
    ax.vlines(24, -2.5, 2.5)
    ax.vlines(30, -2.5, 2.5)
    ax.vlines(36, -2.5, 2.5)
    ax.vlines(42, -2.5, 2.5)
 
    if station_df.shape[0] == 42:
        my_xticks = ['Mon','Tues','Wed','Thurs', 'Fri', 'Sat', 'Sun']
        plt.xticks([3, 9, 15, 21, 27, 33, 39, 45], my_xticks)
        plt.xlim([0, 41])
    else:
        my_xticks = ['Mon-Thurs', 'Fri', 'Sat', 'Sun']
        plt.xticks([3, 9, 15, 21], my_xticks)
        plt.xlim([0, 23])
    
    ax.axes.set_ylim=[-2.5, 2.5] 
    plt.xlabel('Time', fontsize=20)
    plt.ylabel('Count (Standardized)', fontsize=20)

    save_name = plot_name.replace(' ', '')
    save_name = save_name.replace('/', '-')
    save_name = save_name.replace(':', '-')
    plt.savefig(main_path + '/Results/Pics/' + folder_name + '/' + save_name + '.png', dpi=300)
    plt.close()

    
for this_station in df_1_by_wkday.STATION.unique():
    plot_data_by_station_normed(df_1_by_wkday.set_index('STATION'), this_station, 
                                nyc_subway_trnstl_template, 'AllDays')
    
for this_station in df_1_by_wkday_type.STATION.unique():
    plot_data_by_station_normed(df_1_by_wkday_type.set_index('STATION'), this_station, 
                                nyc_subway_trnstl_template, 'BlkDays')

In [36]:
# plot 1 station with only entries first (for presentation)
this_station = 'BAY RIDGE AV (R)'
this_df = df_1_by_wkday.copy()
this_df.loc[this_df.loc[:, 'STATION'] == this_station, 'EXITS'] = np.nan
plot_data_by_station_normed(this_df.set_index('STATION'), this_station, 
                                nyc_subway_trnstl_template, 'AllDaysPreso')

In [37]:
# get map of blocks
nyc_blk_map = gpd.read_file(main_path + '/Data/Locations/2010CensusBlocks/'
                            'geo_export_8ec073df-ddd1-44a2-9db8-f3c2bc37058c.shp')
nyc_blk_map = nyc_blk_map[nyc_blk_map.boro_name.isin(['Brooklyn', 
                                                        'Queens', 'Bronx', 'Staten Island'])]
nyc_blk_map = nyc_blk_map.reset_index() # need to do this
nyc_blk_map['Block'] = nyc_blk_map['bctcb2010'].apply(lambda x: str(x))
nyc_blk_map = nyc_blk_map[['Block', 'geometry' ]]
nyc_blk_map['Centroid'] = nyc_blk_map.centroid
nyc_blk_map['LONG/LAT'] = [(c.x, c.y) for c in nyc_blk_map['Centroid']]

# now for subway map template
nyc_subway_trnstl_template = nyc_subway_trnstl_template.drop('index_right', axis=1)
nyc_subway_trnstl_template = nyc_subway_trnstl_template.reset_index()
nyc_subway_trnstl_template['LONG/LAT'] = [(c.x, c.y) for c in nyc_subway_trnstl_template['geometry']]

In [38]:
# spherical distance
# Taken from here: 
# https://community.esri.com/groups/coordinate-reference-systems/blog/2017/10/05/haversine-formula
def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 3)
    km = round(km, 3)
    miles = round(meters/1609.34,3)
    
    return miles
    #print(f"Distance: {meters} m")
    #print(f"Distance: {km} km")

In [39]:
# Make distance matrix
# REL: need to vectorize this
nyc_blk_map = nyc_blk_map.reset_index()
num_rows = nyc_blk_map.shape[0]

# limit subway template to values that survived
nyc_subway_trnstl_template = nyc_subway_trnstl_template[nyc_subway_trnstl_template \
                                                        .STATION.isin(df_1_by_wkday_type.STATION)]
nyc_subway_trnstl_template = nyc_subway_trnstl_template.reset_index()
num_cols = nyc_subway_trnstl_template.shape[0]

dist_mat = np.zeros([num_rows, num_cols])
for i in range(0, num_rows):
    for j in range(0, num_cols):
        dist_mat[i,j] = haversine(nyc_blk_map.loc[i, 'LONG/LAT'], nyc_subway_trnstl_template.loc[j, 'LONG/LAT'])

In [40]:
# filter each block to subways within 3/4 of a mile
# this might still have out of borough bleed over
dist_mat_filt = 0.75 - dist_mat
dist_mat_filt[dist_mat_filt < 0] = 0

In [41]:
# save stuff
df_1_by_wkday.to_csv(main_path + '/Results/df_by_wkday.csv')
df_1_by_wkday_type.to_csv(main_path + '/Results/df_by_wkday_type.csv')
np.savetxt(main_path + '/Results/dist_block_to_station.csv', dist_mat_filt, delimiter=",")
nyc_subway_trnstl_template.to_csv(main_path + '/Results/subway_station_info.csv')
nyc_blk_map = nyc_blk_map.drop(['Centroid', 'index', 'LONG/LAT'], axis=1)
nyc_blk_map.to_file(main_path + '/Results/nyc_blk_map.shp')

/Users/rita/anaconda3/envs/insight/lib/python3.7/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
